# Recommendation Systems for Hearthstone Decks

In [3]:
!pip install pyreclab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 4.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import pyreclab
import seaborn as sns
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt

In [7]:
url = 'https://raw.githubusercontent.com/gustavo-cornejo/RecSist-Proyecto/refs/heads/main/archive/data.csv?token=GHSAT0AAAAAACYQMCVXGQUFA5SZ7VYGB3EQZYIIMYQ'
data = pd.read_csv(url)
data.head()

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
0,9740,2016-02-19,Unknown,Priest,W,433004,Explorers,Tavern Brawl,1,Reno Priest,...,374,2280,2511,2555,2566,2582,2683,2736,2568,2883
1,9840,2016-02-19,Unknown,Warrior,W,433003,Explorers,Ranked Deck,1,RoosterWarrior,...,1781,1781,2021,2021,2064,2064,2078,2510,2729,2736
2,2600,2016-02-19,Unknown,Mage,W,433002,Explorers,Theorycraft,1,Annoying,...,1793,1801,1801,2037,2037,2064,2064,2078,38710,38710
3,15600,2016-02-19,Unknown,Warrior,W,433001,Explorers,NaN,0,Standart pay to win warrior,...,1657,1721,2018,2296,2262,336,2729,2729,2736,2760
4,7700,2016-02-19,Unknown,Paladin,W,432997,Explorers,Ranked Deck,1,Palamix,...,2027,2029,2029,2064,2078,374,2717,2717,2889,2889


## Random

In [10]:
def random():
  return data.sample(n=1)
random()

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
313476,2420,2015-09-19,Unknown,Druid,W,333966,TGT Launch,Theorycraft,1,Token druid,...,1781,1802,2278,2278,2295,2295,2503,2503,2792,2792


## Most Popular

In [9]:
def most_popular():
  return data[data['rating'] == data['rating'].max()]
most_popular()

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
209833,13520,2015-10-15,Unknown,Warrior,W,347793,TGT Launch,Theorycraft,4016,Legend of the Warsong Commander,...,932,993,1009,1037,1147,1372,556,1135,1721,1910


## Content-Based Filtering

In [50]:
def best_similar(deck_id):
  my_type = data[data['deck_id'] == deck_id].deck_type.values[0]
  similar_data = data[data['deck_type'] == my_type]
  best_similar = similar_data[similar_data['rating'] == similar_data['rating'].max()]
  return best_similar

In [51]:
best_similar(433003)

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
293743,3000,2015-04-07,Patron Warrior,Warrior,S,224656,Blackrock Launch,Ranked Deck,1811,Senfglas Arcane Giant Patron OTK,...,1659,344,2279,2279,38530,38530,38918,38918,39426,39426


In [22]:
def difference(id_1, id_2):
  deck_1 = data[data['deck_id'] == id_1].iloc[:, 11:41].values.flatten()
  deck_2 = data[data['deck_id'] == id_2].iloc[:, 11:41].values.flatten()
  common = len(np.intersect1d(deck_1, deck_2))
  diff = len(deck_1) - common
  return diff